# Utilisation des outils
Dans cet exercice, nous allons franchir une étape clé dans la construction d’un agent intelligent : permettre au modèle de langage d’interagir avec des outils externes. Jusqu’à présent, le LLM se contentait de raisonner et de produire du texte. Or, pour répondre à des questions ancrées dans le monde réel (date actuelle, météo, données externes…), il est nécessaire de lui donner accès à des fonctions capables d’effectuer des actions concrètes.

## Définition des variables
Les variables sont lues depuis le fichier [.env](../../.env)

In [1]:
from dotenv import dotenv_values
config = dotenv_values("../../.env")

llm_model = config.get('LLM_MODEL')
api_key = config.get('LLM_API_KEY')
# Uncomment for local api call
# api_base = config.get('LLM_API_URL')

## Configuration du llm sur dspy

In [2]:
import dspy

lm = dspy.LM(llm_model, api_key=api_key)
# Uncomment for local api call
#lm = dspy.LM(llm_model, api_base=api_base, track_usage=True, temperature=1.5, max_tokens=1024)

dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)
dspy.configure(lm=lm)

<ins>**Exercice:**</ins> construire un agent capable de répondre à une question utilisateur du type : « Quelle est la température actuelle pour la ville de Douala ? » 

## Création des outils
Pour créer une fonction utilisable comme outil dans DSPy, il suffit de définir une fonction Python classique avec une signature claire et des paramètres typés, puis de lui associer une docstring explicite décrivant précisément son rôle. DSPy s’appuie sur cette signature et sur la documentation pour comprendre quand et comment appeler l’outil au cours du raisonnement de l’agent.

Il vous faudra donc déterminer les outils nécessaires à créer pour répondre aux besoins.

<details>
    <summary>Voir plus d'indices</summary>
    Trois fonctions sont nécessaires:
    <ul>
        <li><code>get_current_datetime</code>: Cette fonction retourne la date au format iso et permettra au llm d'avoir l'information de la date et heure courante au moment de l'exécution, pour ce faire vous pouvez utiliser la librarie `datetime` de python.</li>
        <li><code>get_geolocalisation</code> : Cette fonction prend en entrée le nom d’une ville (<code>town_name</code>) et permet au LLM d'obtenir sa position géographique. Elle interroge l'API de géocodage d'Open-Meteo (<code>f'https://geocoding-api.open-meteo.com/v1/search/v1/search</code>) avec les paramètres <code>name</code> et <code>count=1</code>, puis retourne la latitude et la longitude du premier résultat.
        </li>
        <li><code>get_temperature_forecast</code> : Cette fonction utilise les coordonnées géographiques (<code>latitude</code> et <code>longitude</code>) pour récupérer les prévisions de température. Elle appelle l'API de prévisions météo d’Open-Meteo (<code>f'https://api.open-meteo.com/v1/forecast/v1/forecast</code>) en précisant les paramètres <code>hourly=temperature_2m</code>, <code>forecast_days=1</code> et <code>format=json</code>. La fonction retourne une liste structurée associant chaque date ou heure à la température correspondante, afin que le LLM puisse facilement exploiter ces données dans sa réponse finale.
        </li>
    </ul>
</details>

In [3]:
import requests
from datetime import datetime
                         
class ForecastTools():
    """Forecast set of tools"""

    def get_current_datetime(self):
        """Current datetime"""
        return datetime.today().isoformat()
    
    def get_geolocalisation(self, town_name: str):
        """Return city geolocation: latitude and longitude"""
        response = requests.get(f'https://geocoding-api.open-meteo.com/v1/search?count=1&name={town_name}')
        return {
            'longitude': response.json()['results'][0]['longitude'],
            'latitude': response.json()['results'][0]['latitude']
        }
    
    def get_temperature_forecast(self, latitude: float, longitude: float):
        """Get daily temperature forecast of given latitude and longitude"""
        response = requests.get(f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m&forecast_days=1&format=json')
        date_serie = response.json()['hourly']['time']
        temperature_serie = response.json()['hourly']['temperature_2m']
        result = []
        for i in range(len(date_serie)):
            result.append({
                'datetime_unix': date_serie[i],
                'temperature': temperature_serie[i]
            })
        return result
    

## Créer une prédiction basée sur les outils
Quel module DSPy permet de créer ce type de traitement ? Trouvez la réponse à cette question et adaptez le code suivant

In [5]:
tools = ForecastTools()
assistant = dspy.ReAct(
    "question -> answer",
    tools=[
        tools.get_current_datetime,
        tools.get_geolocalisation,
        tools.get_temperature_forecast
    ]
)

## Exécution

In [6]:
from rich import print

response = assistant(question="Quelle est la température actuelle pour la ville de Douala ?")

print(response)
print(lm.history)
dspy.inspect_history()

Prediction(
    trajectory={'thought_0': "Je dois d'abord localiser la ville de Douala pour obtenir ses coordonnées 
géographiques, puis je pourrai demander la température actuelle pour cette localisation.", 'tool_name_0': 
'get_geolocalisation', 'tool_args_0': {'town_name': 'Douala'}, 'observation_0': {'longitude': 9.70428, 'latitude': 
4.04827}, 'thought_1': 'Now that I have the coordinates of Douala, I can request the current temperature forecast 
using these latitude and longitude values.', 'tool_name_1': 'get_temperature_forecast', 'tool_args_1': {'latitude':
4.04827, 'longitude': 9.70428}, 'observation_1': [{'datetime_unix': '2026-02-05T00:00', 'temperature': 25.0}, 
{'datetime_unix': '2026-02-05T01:00', 'temperature': 24.8}, {'datetime_unix': '2026-02-05T02:00', 'temperature': 
24.6}, {'datetime_unix': '2026-02-05T03:00', 'temperature': 24.6}, {'datetime_unix': '2026-02-05T04:00', 
'temperature': 24.5}, {'datetime_unix': '2026-02-05T05:00', 'temperature': 24.5}, {'datetime_unix': 
'2026-02-05T06:00', 'temperature': 24.2}, {'datetime_unix': '2026-02-05T07:00', 'temperature': 24.6}, 
{'datetime_unix': '2026-02-05T08:00', 'temperature': 25.3}, {'datetime_unix': '2026-02-05T09:00', 'temperature': 
26.2}, {'datetime_unix': '2026-02-05T10:00', 'temperature': 27.3}, {'datetime_unix': '2026-02-05T11:00', 
'temperature': 28.0}, {'datetime_unix': '2026-02-05T12:00', 'temperature': 28.8}, {'datetime_unix': 
'2026-02-05T13:00', 'temperature': 29.3}, {'datetime_unix': '2026-02-05T14:00', 'temperature': 29.4}, 
{'datetime_unix': '2026-02-05T15:00', 'temperature': 28.8}, {'datetime_unix': '2026-02-05T16:00', 'temperature': 
28.3}, {'datetime_unix': '2026-02-05T17:00', 'temperature': 27.8}, {'datetime_unix': '2026-02-05T18:00', 
'temperature': 27.3}, {'datetime_unix': '2026-02-05T19:00', 'temperature': 27.0}, {'datetime_unix': 
'2026-02-05T20:00', 'temperature': 26.9}, {'datetime_unix': '2026-02-05T21:00', 'temperature': 26.8}, 
{'datetime_unix': '2026-02-05T22:00', 'temperature': 26.6}, {'datetime_unix': '2026-02-05T23:00', 'temperature': 
26.6}], 'thought_2': "La température actuelle de Douala, correspondant à l'heure locale, semble être autour de 
26.6°C à la fin de la journée, mais la dernière heure disponible (23:00) montre 26.6°C. Il serait prudent de donner
cette valeur comme l'information actuelle ou récente pour la température.", 'tool_name_2': 'finish', 'tool_args_2':
{}, 'observation_2': 'Completed.'},
    reasoning="D'après les données de la prévision météorologique, la température la plus récente est celle de 
23h00, qui indique 26.6°C. Étant donné que la demande concerne la température actuelle à Douala, la valeur la plus 
récente disponible est la plus pertinente. En tenant compte du contexte et de la météo prévue, il est raisonnable 
d'indiquer que la température actuelle est d'environ 26.6°C.",
    answer="La température actuelle à Douala est d'environ 26,6°C."
)

[
    {
        'prompt': None,
        'messages': [
            {
                'role': 'system',
                'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output 
fields are:\n1. `next_thought` (str): \n2. `next_tool_name` (Literal[\'get_current_datetime\', 
\'get_geolocalisation\', \'get_temperature_forecast\', \'finish\']): \n3. `next_tool_args` (dict[str, Any]):\nAll 
interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## 
]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## next_thought ## ]]\n{next_thought}\n\n[[ ## 
next_tool_name ## ]]\n{next_tool_name}        # note: the value you produce must exactly match (no extra 
characters) one of: get_current_datetime; get_geolocalisation; get_temperature_forecast; finish\n\n[[ ## 
next_tool_args ## ]]\n{next_tool_args}        # note: the value you produce must adhere to the JSON schema: 
{"type": "object", "additionalProperties": true}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your 
objective is: \n        Given the fields `question`, produce the fields `answer`.\n        \n        You are an 
Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so 
far.\n        Your goal is to use one or more of the supplied tools to collect any necessary information for 
producing `answer`.\n        \n        To do this, you will interleave next_thought, next_tool_name, and 
next_tool_args in each turn, and also when finishing the task.\n        After each tool call, you receive a 
resulting observation, which gets appended to your trajectory.\n        \n        When writing next_thought, you 
may reason about the current situation and plan for future steps.\n        When selecting the next_tool_name and 
its next_tool_args, the tool must be one of:\n        \n        (1) get_current_datetime, whose description is 
<desc>Current datetime</desc>. It takes arguments {}.\n        (2) get_geolocalisation, whose description is 
<desc>Return city geolocation: latitude and longitude</desc>. It takes arguments {\'town_name\': {\'type\': 
\'string\'}}.\n        (3) get_temperature_forecast, whose description is <desc>Get daily temperature forecast of 
given latitude and longitude</desc>. It takes arguments {\'latitude\': {\'type\': \'number\'}, \'longitude\': 
{\'type\': \'number\'}}.\n        (4) finish, whose description is <desc>Marks the task as complete. That is, 
signals that all information for producing the outputs, i.e. `answer`, are now available to be extracted.</desc>. 
It takes arguments {}.\n        When providing `next_tool_args`, the value inside the field must be in JSON format'
            },
            {
                'role': 'user',
                'content': "[[ ## question ## ]]\nQuelle est la température actuelle pour la ville de Douala 
?\n\n[[ ## trajectory ## ]]\n\n\nRespond with the corresponding output fields, starting with the field `[[ ## 
next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python 
Literal['get_current_datetime', 'get_geolocalisation', 'get_temperature_forecast', 'finish']), then `[[ ## 
next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[
## completed ## ]]`."
            }
        ],
        'kwargs': {},
        'response': ModelResponse(
            id='chatcmpl-D5zGCt2GpS8lrXUkrQ7GwOWh26v8p',
            created=1770319496,
            model='gpt-4.1-nano-2025-04-14',
            object='chat.completion',
            system_fingerprint='fp_de604bd877',
            choices=[
                Choices(
                    finish_reason='stop',
                    index=0,
                    message=Message(
                        content='[[ ## next_thought ## ]]\nJe dois d\'abord localiser la ville de Douala pour 
obtenir ses coordonnées géographiques, puis je po





[2026-02-05T20:25:03.965891]

System message:

Your input fields are:
1. `question` (str): 
2. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
Quelle est la température actuelle pour la ville de Douala ?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
Je dois d'abord localiser la ville de Douala pour obtenir ses coordonnées géographiques, puis je pourrai demander la température actuelle pour cette localisation.

[[ ## tool_name_0 ## ]]
get_geolocalisation

[[ ## tool_args_0 ## ]]
{"town_name": "Douala"}

[[ ## observation_0 ## ]]
{"longitude": 9